In [1]:
from binance.client import Client
from binance import ThreadedWebsocketManager
import binance as bn
from binance.exceptions import BinanceAPIException
import requests as requests
import sys
import os
import json
from datetime import datetime, timedelta, timezone
import ta as ta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import smtplib
import time
import json
import copy
sys.path.append(os.path.dirname(os.getcwd()))
from utils import utils
from keys import keys

In [2]:
pd.set_option('display.max_rows', None)

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
class Futures_trader():
    """
    Class to perform live testing using Binance Futures testnet stream of data. v1.0 (sl and tp should be working)
    """ 
    def __init__(self, sc_name='BUSD', api_key=None, api_secret=None, testnet=None, symbol=None, units='0.006', interval=None, ema_slow=None, ema_fast=None, ema_signal=None, sma_trend_slow=None, ema_trend_fast=None, sl=None, tp=None, leverage=2, assigned_duration_minutes=None, tp_cont=False):
        """
        :param sc_name: stable coin for pairs
        :type symbol: str.
        ----
        :param api_key: binance API key string
        :type symbol: str.
        ----
        :param api_secret: binance API secret string
        :type symbol: str.
        ----
        :param testnet: if True, testnet is used, otherwise REAL Binance account
        :type testnet: bool.print
        ----
        :param symbol: ticker in Binance, i.e. "BTCBUSD"
        :type symbol: str.
        ----
        :param units: amount of base units, i.e. "BTC"
        :param type: float.
        ----
        :param interval: a string among the followings: ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        :type interval: str.
        ----
        :param ema_slow: EMA slow for MACD calculation
        :type ema_slow: int.
        ----
        :param ema_fast: EMA fast for MACD calculation
        :type ema_fast: int.
        ----
        :param ema_signal: EMA signal for MACD calculation
        :type ema_signal: int.
        ----
        :param sma_trend_slow: SMA slow signal for sma/ema cross calculation
        :type sma_trend_slow: int.
        ----        
        :param ema_trend_fast: EMA fast signal for sma/ema cross calculation
        :type ema_trend_fast: int.
        ----            
        :param sl: per unit of closing price to execute stop loss
        :type sl: float
        ----
        :param tp: per unit of closing price to execute take profith
        :type tp: float
        ----
        :param leverage: leverage to use
        :type leverage: integer
        ---
        :param assigned_duration_minutes: amount of minutes that the sesion is expected to last, if no problems appear.
        :type assigned_duration_minutes: int.
        ---
        :param tp_cont: parameter to trigger the execution of buy/sell order after taking profit (to realize profits in small quantities)
        :type tp_cont: int.
        ----
        ----
        """
        # constructor variables
        self.sc_name = sc_name
        self.api_key = api_key
        self.api_secret = api_secret
        self.testnet = testnet    
        self.symbol = symbol
        self.units = units
        self.interval = interval
        self.ema_slow = ema_slow
        self.ema_fast = ema_fast
        self.ema_signal = ema_signal 
        self.sma_trend_slow = sma_trend_slow
        self.ema_trend_fast = ema_trend_fast
        self.sl = sl
        self.tp = tp
        self.leverage = leverage
        self.assigned_duration_minutes = assigned_duration_minutes
        
        # other variables
        self.run_end_time_utc = None # time in UTC when the calculation finished
        self.run_end_delta = None # amount of time that has passed since the beginning of the calculation
        self.data = pd.DataFrame() # dataframe to contain all OHLC data
        self.orders = 0 # counter of the number of orders
        self.client = Client(api_key=self.api_key, api_secret=self.api_secret, tld = "com", testnet = self.testnet) # Binance client initialization
        self.trade_start_time_utc = None # time in utc to be defined when the stream of OHLC starts ( this time
        self.twm = None # websockets client
        self.cum_profits = 0 # accumulated profits in the trading sesion
        self.conn = None # smtp connection
        self.login_mail() # initialize smtp google account
        self.td = self.calculate_td() # initialize delta t depending on interval chosen
        self.orders_list = [] # list of orders
        self.trades_list = [] # list of trades (an order can contain multiple trades)
        self.position = 0 # initial position
        self.last_order = None
        self.sl_flag_short = None # flag for stop loss/ take profit algorithm
        self.tp_flag_short = None # flag for stop loss/ take profit algorithm
        self.sl_flag_long = None # flag for stop loss/ take profit algorithm
        self.tp_flag_long = None # flag for stop loss/ take profit algorithm
        self.sltp_flag = None # flag for stop loss/ take profit algorithm
        self.sltp_tini = None # starting time for stop loss/take profit calculation
        self.sltp_pini = None # instantaneus price registered at the time the order is placed
        self.is_initial = True # flag to mark if the current position has been stablished by prepare_recent_data it is set to False once tpdate_position() is executed for the first time
        self.tp_cont = tp_cont # flag to activate the take profit continuation
        self.sltp_pend_list = [] # list of stop loss / take profit trigger prices
        self.tevent_list = [] # list of times of stop loss / take profit trigger prices
        self.balance_initial = self.get_actual_balance() # get initial wallet balance
        self.balance_end = None # final balance at the end of the sesion
        self.balance_profit_end = None # final profit at the end of the sesion
        self.balance_actual = None # actual balance in the current sesion time
        self.balance_profit_actual = None # actual profit in the current sesion time
    
    def calculate_td(self):
        '''
        Calculate time delta used for finding the data of the inmediate kandle prior to the current one
        '''
        td = timedelta()
        if 'm' in self.interval:
            num_min = int(self.interval.replace('m',''))
            td = timedelta(minutes=num_min)
        if 'h' in self.interval:
            num_h = int(self.interval.replace('h',''))
            td = timedelta(hours=num_h)
        if 'd' in self.interval:
            num_day = int(self.interval.replace('d',''))
            td = timedelta(days=num_day)
        if 'w' in self.interval:
            num_week = int(self.interval.replace('w',''))
            td = timedelta(weeks=num_week)
        if 'M' in self.interval:
            num_week_m = int(self.interval.replace('M',''))
            td = timedelta(weeks=4*num_week)
        return td
    
    def remake_client(self):
        '''
        Remake Binance client in certain cases where an error occurs
        '''
        self.client = Client(api_key=self.api_key, api_secret=self.api_secret, tld = "com", testnet = self.testnet)

    def handle_exception(self, error=None, fname=None):
        '''
        Stop the current sesion if any type of error occurs.
        '''
        print(f"except in {fname}")
        print(f"Something went wrong. Error: {error} occured at t: {datetime.now().astimezone(timezone.utc)}. The sesion will be automatically stopped after GOING NEUTRAL.")
        self.stop_ses()
    
    def start_trading(self):
        '''
        1) calculate start time
        2) prepare historical data in order to have enought data for the moving averages windows when sesion starts
        3) calculate returns initially (at this moment this is an approximation)
        4) initialize websockets
        '''
        try:
            self.trade_start_time_utc = datetime.utcnow()
            self.prepare_recent_data()
            self.stablish_initial_position()
            self.calculate_returns()
            self.client.futures_change_leverage(symbol = self.symbol, leverage = self.leverage) #initialize leverage 
            self.init_socket()            
        except Exception as e:
            self.handle_exception(e, 'start_trading')
        
    def init_socket(self):
        '''
        start streaming data
        '''
        try:
            self.twm = ThreadedWebsocketManager()
            self.twm.start()
            self.twm.start_kline_futures_socket(callback = self.stream_candles, symbol = self.symbol, interval = self.interval)
        except Exception as e:
            self.handle_exception(e, 'init_socket')

    def stream_candles(self, msg):
        '''
        1) format data coming from the Binance stream
        2) initialize macd/sma/ema
        3) stop sesion when assigned time has passed
        4) execute stop loss/take profit (sltp) when conditions are met regardless of kandle completion
        5) when kandle completion update position, calculate returns, update balance and execute trades
        '''
        try:
            event_time = pd.to_datetime(msg["E"], unit = "ms")
            start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
            self.klast = start_time # latest kandle date, complete or incomplete
            self.kprev = start_time - self.td
            self.tevent = event_time
            first   = float(msg["k"]["o"])
            high    = float(msg["k"]["h"])
            low     = float(msg["k"]["l"])
            close   = float(msg["k"]["c"])
            volume  = float(msg["k"]["v"])
            complete=       msg["k"]["x"]
            # feed df (add new bar / update latest bar)
            self.data.loc[start_time, 'Open'] = first
            self.data.loc[start_time, 'High'] = high
            self.data.loc[start_time, 'Low'] = low
            self.data.loc[start_time, 'Close'] = close
            self.data.loc[start_time, 'Volume'] = volume
            self.data.loc[start_time, 'Complete'] = complete
            # update MACD parameters with each ws retrieval
            macd_diff = ta.trend.MACD(close=self.data.Close, window_slow=self.ema_slow, window_fast=self.ema_fast, window_sign=self.ema_signal, fillna=False).macd_diff()
            macd_macd = ta.trend.MACD(close=self.data.Close, window_slow=self.ema_slow, window_fast=self.ema_fast, window_sign=self.ema_signal, fillna=False).macd()
            macd_signal = ta.trend.MACD(close=self.data.Close, window_slow=self.ema_slow, window_fast=self.ema_fast, window_sign=self.ema_signal, fillna=False).macd_signal()         
            # assigning the macd values to ticker dataframe
            self.data.loc[start_time, 'macd_diff'] = macd_diff.iloc[-1]
            self.data.loc[start_time, 'macd_macd'] = macd_macd.iloc[-1]
            self.data.loc[start_time, 'macd_signal'] = macd_signal.iloc[-1]
            # update SMAs with each ws retrieval
            ema_trend_fast_ps = ta.trend.EMAIndicator(close=self.data.Close, window=self.ema_trend_fast, fillna=False).ema_indicator()
            sma_trend_slow_ps = ta.trend.SMAIndicator(close=self.data.Close, window=self.sma_trend_slow, fillna=False).sma_indicator()
            # assigning the smas values to ticker dataframe
            self.data.loc[start_time, 'ema_trend_fast_ps'] = ema_trend_fast_ps.iloc[-1]
            self.data.loc[start_time, 'sma_trend_slow_ps'] = sma_trend_slow_ps.iloc[-1]

            # print(f"event_time = {event_time}, start_time = {start_time}, complete = {complete}, ema_trend_fast_ps = {ema_trend_fast_ps.iloc[-1]}, sma_trend_slow_ps = {sma_trend_slow_ps.iloc[-1]}")
            print('.', end ='', flush = True) # just print something to get a feedback (everything OK)        

            dt = datetime.utcnow() - self.trade_start_time_utc
            if ((dt) > timedelta(minutes=self.assigned_duration_minutes)):
                self.stop_ses()

            if (self.is_initial == False and self.sltp_tini != None):    
                self.assess_sltp()                
            
            if complete == True:
                self.update_position()
                self.calculate_returns() 
                self.update_balance_sc() # so that there are no NaN when position is not changed
                self.execute_trades()           
            else:
                pass
        except Exception as e:
            self.handle_exception(e, 'stream_candles')        
        
    def prepare_recent_data(self):
        '''
        Prepare historial data to be used in the initialization of technical indicators
        '''
        try:
            current_time_obj = datetime.now()
            current_time = int(current_time_obj.timestamp()*1000)
            extra_periods = 0
            ema_diff = self.ema_slow + self.ema_signal + 2
            if (ema_diff >= self.sma_trend_slow):
                extra_periods = ema_diff * 2
            else:
                extra_periods = self.sma_trend_slow * 2

            td = timedelta()
            if 'm' in self.interval:
                num_min = int(self.interval.replace('m',''))
                td = timedelta(minutes=num_min*(extra_periods))
            if 'h' in self.interval:
                num_h = int(self.interval.replace('h',''))
                td = timedelta(hours=num_h*(extra_periods))
            if 'd' in self.interval:
                num_day = int(self.interval.replace('d',''))
                td = timedelta(days=num_day*(extra_periods))
            if 'w' in self.interval:
                num_week = int(self.interval.replace('w',''))
                td = timedelta(weeks=num_week*(extra_periods))
            if 'M' in self.interval:
                num_week_m = int(self.interval.replace('M',''))
                td = timedelta(weeks=num_week_m * 4 * (extra_periods))

            from_time_obj = current_time_obj - td
            from_time = int((current_time_obj - td).timestamp()*1000)
            # start/end: a string with the following format ""%Y-%m-%d-%H:%M" .i.e. "2022-01-29-20:00"
            self.data = utils.futures_history(symbol=self.symbol, interval=self.interval, start=from_time, end=current_time, testnet=self.testnet)
            # obtaining MACD instance from python ta
            macd_diff = ta.trend.MACD(close=self.data.Close, window_slow=self.ema_slow, window_fast=self.ema_fast, window_sign=self.ema_signal, fillna=False).macd_diff()
            macd_macd = ta.trend.MACD(close=self.data.Close, window_slow=self.ema_slow, window_fast=self.ema_fast, window_sign=self.ema_signal, fillna=False).macd()
            macd_signal = ta.trend.MACD(close=self.data.Close, window_slow=self.ema_slow, window_fast=self.ema_fast, window_sign=self.ema_signal, fillna=False).macd_signal()
            # assigning the values of macd to ticker dataframe
            self.data['macd_diff'] = macd_diff
            self.data['macd_macd'] = macd_macd
            self.data['macd_signal'] = macd_signal
            # calculating SMA fast and slow
            ema_trend_fast_ps = ta.trend.EMAIndicator(close=self.data.Close, window=self.ema_trend_fast, fillna=False).ema_indicator()
            sma_trend_slow_ps = ta.trend.SMAIndicator(close=self.data.Close, window=self.sma_trend_slow, fillna=False).sma_indicator()
            # assigning the smas values to ticker dataframe
            self.data['ema_trend_fast_ps'] = ema_trend_fast_ps
            self.data['sma_trend_slow_ps'] = sma_trend_slow_ps
            # initialize dataframe columns for prepared data
            self.data['position'] = 0
            self.data['macd_inv_sign'] = 0
            self.data['sma_conf'] = 0
            self.data['sma_inv_sign'] = 0
            self.data['macd_diff_conf'] = 0
            if (self.tp_cont == True):
                self.data['tp_cont'] = None
            if (self.sl != None and self.tp != None):
                self.data['sl_flag_short'] = False
                self.data['tp_flag_short'] = False
                self.data['sl_flag_long'] = False
                self.data['tp_flag_long'] = False
                self.data['sltp_tini'] = None
                self.data['sltp_tend'] = None
                self.data['sltp_pini'] = None
                self.data['sltp_pend'] = None 
            else: 
                pass
            self.data["Complete"] = [True for row in range(len(self.data)-1)] + [False]
        except Exception as e:
            self.handle_exception(e, 'prepare_recent_data') 

    def stablish_initial_position(self): 
        '''
        Stablish initial positions based of historical data
        '''
        try:
            # --------------------- MACD CROSS SIGNAL + SMA CONFIRMATION ---------------------

            # macd cross short
            ht_macd_diff_pos = self.data.macd_diff.shift(1) > 0
            ht_macd_diff_plusone_neg = self.data.macd_diff < 0        

            # sma short confirmation
            sma_conf_short = self.data.ema_trend_fast_ps < self.data.sma_trend_slow_ps

            # stablish short positions
            self.data.loc[ht_macd_diff_pos & ht_macd_diff_plusone_neg, 'macd_inv_sign'] = -1
            self.data.loc[sma_conf_short, 'sma_conf'] = -1

            # macd cross long
            ht_macd_diff_neg = self.data.macd_diff.shift(1) < 0
            ht_macd_diff_plusone_pos = self.data.macd_diff > 0

            # sma long confirmation
            sma_conf_long = self.data.ema_trend_fast_ps > self.data.sma_trend_slow_ps        

            # stablish buy positions
            self.data.loc[ht_macd_diff_neg & ht_macd_diff_plusone_pos, 'macd_inv_sign'] = 1
            self.data.loc[sma_conf_long, 'sma_conf'] = 1

            # ---------------------------------------------------------------------------------

            # --------------------- SMA CROSS SIGNAL + MACD_DIFF CONFIRMATION -----------------

            # sma cross short
            ht_sma_pos = self.data.ema_trend_fast_ps.shift(1) > self.data.sma_trend_slow_ps.shift(1)
            ht_sma_plusone_neg = self.data.ema_trend_fast_ps < self.data.sma_trend_slow_ps

            # macd_diff short confirmation
            macd_diff_conf_short = self.data.macd_diff < 0

            # stablish short positions
            self.data.loc[ht_sma_pos & ht_sma_plusone_neg, 'sma_inv_sign'] = -1
            self.data.loc[macd_diff_conf_short, 'macd_diff_conf'] = -1

            # sma cross long
            ht_sma_neg = self.data.ema_trend_fast_ps.shift(1) < self.data.sma_trend_slow_ps.shift(1)
            ht_sma_plusone_pos = self.data.ema_trend_fast_ps > self.data.sma_trend_slow_ps        

            # macd_diff long confirmation
            macd_diff_conf_long = self.data.macd_diff > 0

            # stablish buy positions
            self.data.loc[ht_sma_neg & ht_sma_plusone_pos, 'sma_inv_sign'] = 1
            self.data.loc[macd_diff_conf_long, 'macd_diff_conf'] = 1        

            # ---------------------------------------------------------------------------------
            for index, data in self.data.iterrows():

                condlong1 = ((data.macd_inv_sign == 1 and data.sma_inv_sign == 1) or (data.macd_inv_sign == 1 and data.sma_conf == 1))
                condshort1 = ((data.macd_inv_sign == -1 and data.sma_inv_sign == -1) or (data.macd_inv_sign == -1 and data.sma_conf == -1))
                condlong2 = ((data.sma_inv_sign == 1 and data.macd_inv_sign == 1) or (data.sma_inv_sign == 1 and data.macd_diff_conf == 1))
                condshort2 = ((data.sma_inv_sign == -1 and data.macd_inv_sign == -1) or (data.sma_inv_sign == -1 and data.macd_diff_conf == -1))

                if (condlong1 == True and condshort1 == False and condlong2 == False and condshort2 == False):
                    self.data.loc[index, 'position'] = 1
                    one_delta_pos = index + (self.data.index[1]-self.data.index[0])
                    self.data_sub_buy = self.data.loc[one_delta_pos:]
                    for index_sub_buy, data_sub_buy in self.data_sub_buy.iterrows():
                        condsubshort1 = ((data_sub_buy.macd_inv_sign == -1 and data_sub_buy.sma_inv_sign == -1) or (data_sub_buy.macd_inv_sign == -1 and data_sub_buy.sma_conf == -1))
                        condsubshort2 = ((data_sub_buy.sma_inv_sign == -1 and data_sub_buy.macd_inv_sign == -1) or (data_sub_buy.sma_inv_sign == -1 and data_sub_buy.macd_diff_conf == -1))                                                       
                        if (condsubshort1 == True or condsubshort2 == True):
                            break
                        else: 
                            self.data.loc[index_sub_buy, 'position'] = 1

                elif (condlong1 == False and condshort1 == True and condlong2 == False and condshort2 == False): 
                    self.data.loc[index, 'position'] = -1
                    one_delta_pos = index + (self.data.index[1]-self.data.index[0])
                    self.data_sub_sell = self.data.loc[one_delta_pos:]
                    for index_sub_sell, data_sub_sell in self.data_sub_sell.iterrows():
                        condsublong1 = ((data_sub_sell.macd_inv_sign == 1 and data_sub_sell.sma_inv_sign == 1) or (data_sub_sell.macd_inv_sign == 1 and data_sub_sell.sma_conf == 1))
                        condsublong2 = ((data_sub_sell.sma_inv_sign == 1 and data_sub_sell.macd_inv_sign == 1) or (data_sub_sell.sma_inv_sign == 1 and data_sub_sell.macd_diff_conf == 1))                    
                        if (condsublong1 == True or condsublong2 == True):
                            break
                        else:
                            self.data.loc[index_sub_sell, 'position'] = -1

                elif (condlong1 == False and condshort1 == False and condlong2 == True and condshort2 == False):
                    self.data.loc[index, 'position'] = 1
                    one_delta_pos = index + (self.data.index[1]-self.data.index[0])
                    self.data_sub_buy = self.data.loc[one_delta_pos:]
                    for index_sub_buy, data_sub_buy in self.data_sub_buy.iterrows():
                        condsubshort1 = ((data_sub_buy.macd_inv_sign == -1 and data_sub_buy.sma_inv_sign == -1) or (data_sub_buy.macd_inv_sign == -1 and data_sub_buy.sma_conf == -1))
                        condsubshort2 = ((data_sub_buy.sma_inv_sign == -1 and data_sub_buy.macd_inv_sign == -1) or (data_sub_buy.sma_inv_sign == -1 and data_sub_buy.macd_diff_conf == -1))                      
                        if (condsubshort1 == True or condsubshort2 == True):
                            break
                        else:
                            self.data.loc[index_sub_buy, 'position'] = 1

                elif (condlong1 == False and condshort1 == False and condlong2 == False and condshort2 == True):
                    self.data.loc[index, 'position'] = -1
                    one_delta_pos = index + (self.data.index[1]-self.data.index[0])
                    self.data_sub_sell = self.data.loc[one_delta_pos:]
                    for index_sub_sell, data_sub_sell in self.data_sub_sell.iterrows():
                        condsublong1 = ((data_sub_sell.macd_inv_sign == 1 and data_sub_sell.sma_inv_sign == 1) or (data_sub_sell.macd_inv_sign == 1 and data_sub_sell.sma_conf == 1))
                        condsublong2 = ((data_sub_sell.sma_inv_sign == 1 and data_sub_sell.macd_inv_sign == 1) or (data_sub_sell.sma_inv_sign == 1 and data_sub_sell.macd_diff_conf == 1))                    
                        if (condsublong1 == True or condsublong2 == True):
                            break
                        else:
                            self.data.loc[index_sub_sell, 'position'] = -1 
                else:
                    pass   
            self.kprev = self.data.index[-2]         
            self.position = 0                 
        except Exception as e:
            self.handle_exception(e, 'stablish_initial_position') 
              
    def calculate_returns(self):
        '''
        Calculate returns based on historical data'''
        try:
            # stablish the trading costs and the number of trades done
            data = self.data.loc[self.trade_start_time_utc-self.td:].copy()
            data['trades'] = 0
            trading_cost_fees = 0.0004
            trading_cost_others = 0.00001
            trading_cost = np.log(1 - trading_cost_fees) + np.log(1 - trading_cost_others)
            trade_exec_cond = data.position.diff().fillna(0).abs() != 0
            data.loc[trade_exec_cond, 'trades'] = 1
            total_trades = data.trades[data.trades == 1].sum()
            # calculate strategy returns
            data['log_returns_hold'] = np.log(data.Close.div(data.Close.shift(1)))
            data['multiple_hold_acum'] = np.exp(data.log_returns_hold.cumsum())
            data['macd_log_returns'] = data.log_returns_hold * data.position.shift(1)
            data['macd_log_returns_net'] = data.macd_log_returns + data.trades * trading_cost
            data['macd_multiple_acum'] = np.exp(data.macd_log_returns.cumsum())
            data['macd_multiple_net_acum'] = np.exp(data.macd_log_returns_net.cumsum())
            # calculating profitable trades with trading costs
            cond_profit_net1 = data.trades == 1
            cond_profit_net2 = ((data.Close.div(data.Close.shift(-1)) - 1) + np.log(1 - 0.0004) + np.log(1 - 0.0001))  > 0
            success_trades_net = data[cond_profit_net1&cond_profit_net2].trades.sum()
            if (total_trades != 0):
                success_trades_net_pct = round((success_trades_net / total_trades)*100,1)
            # calculating failure trades with trading costs
            cond_failure_net1 = data.trades == 1
            cond_failure_net2 = ((data.Close.div(data.Close.shift(-1)) - 1) + np.log(1 - 0.00075) + np.log(1 - 0.0001))  < 0
            failure_trades_net = data[cond_failure_net1&cond_failure_net2].trades.sum()
            if (total_trades != 0):
                failure_trades_net_pct = round((failure_trades_net / total_trades)*100, 1)
            # adds/Labels trading sessions and their compound returns.
            data["ses"] = np.sign(data.trades).cumsum().shift().fillna(0)
            data["macd_simplecret_net_ses"] = data.groupby("ses").macd_log_returns_net.cumsum().apply(np.exp) - 1        
            data["macd_simple_ret"] = np.exp(data.macd_log_returns_net) - 1
            data["eff_lev"] = self.leverage * (1 + data.macd_simplecret_net_ses) / (1 + data.macd_simplecret_net_ses * self.leverage)
            data.eff_lev.fillna(self.leverage, inplace = True)
            levered_returns = data.eff_lev.shift() * data.macd_simple_ret
            levered_returns = np.where(levered_returns < -1, -1, levered_returns)
            data["macd_simple_ret_net_lev"] = levered_returns
            data["macd_simple_ret_net_lev_acum"] = data.macd_simple_ret_net_lev.add(1).cumprod()
            self.data["macd_simple_ret_net_lev_acum"] = data["macd_simple_ret_net_lev_acum"]
            self.data['multiple_hold_acum'] = data['multiple_hold_acum']
            self.data['macd_multiple_net_acum'] = data['macd_multiple_net_acum']
        except Exception as e:
            self.handle_exception(e, 'calculate_returns')    
            
    def assess_sltp(self):
        '''
        Assess if a stop loss/take profit signal has to be triggered
        '''
        try:
            # just for convenience
            klast = self.klast
            kprev = self.kprev
            # check if position in previous kandle is short and if so, analize sltp condition
            if (self.data.position.loc[kprev] == -1):
                # condition for take profit in short position
#                 print('tp_flag_short assessment')
#                 print('self.data.Close.loc[klast]', self.data.Close.loc[klast])
#                 print('self.sltp_pini', self.sltp_pini)
#                 print('self.sltp_pini * (1 - self.tp)', self.sltp_pini * (1 - self.tp))
#                 print('self.data.Close.loc[klast] <= self.sltp_pini * (1 - self.tp)', self.data.Close.loc[klast] <= self.sltp_pini * (1 - self.tp))
#                 print(((self.data.Close.loc[klast]/self.sltp_pini)-1), self.tp)
                self.tp_flag_short = self.data.Close.loc[klast] <= self.sltp_pini * (1 - self.tp) 
#                 print('self.tp_flag_short', self.tp_flag_short)               
               
                #condition for stop loss in short position                  
#                 print('sl_flag_short assessment')
#                 print('self.data.Close.loc[klast]', self.data.Close.loc[klast])
#                 print('self.sltp_pini', self.sltp_pini)
#                 print('self.sltp_pini * (1 + self.sl)', self.sltp_pini * (1 + self.sl))
#                 print('self.data.Close.loc[klast] >= self.sltp_pini * (1 + self.sl)', self.data.Close.loc[klast] >= self.sltp_pini * (1 + self.sl))              
#                 print(((self.data.Close.loc[klast]/self.sltp_pini)-1), self.sl)
                self.sl_flag_short = self.data.Close.loc[klast] >= self.sltp_pini * (1 + self.sl)
#                 print('self.sl_flag_short', self.sl_flag_short)                  
                
                if self.tp_flag_short:             
                    self.data.loc[klast, 'tp_flag_short'] = 'tp_flag_short'
                    self.sltp_flag = True
                elif self.sl_flag_short:
                    self.data.loc[klast, 'sl_flag_short'] = 'sl_flag_short'  
                    self.sltp_flag = True                        
                else:
                    self.sltp_flag = False # if no sltp condition it is necessary to reset this variable to not jumpt accidentaly into executing a new trade

                if (self.sltp_flag):
                    self.data.loc[klast, 'position'] = 0
                    # register when the order was placed for the current sltp event
                    self.data.loc[klast, 'sltp_tini'] = self.sltp_tini
                    # register when the position ends due to sltp
                    self.data.loc[klast, 'sltp_tend'] = self.tevent
                    # create a list with all the instants when sltp was triggered
                    self.tevent_list.append(self.tevent)
                    # register price when the order was placed for the current sltp event
                    self.data.loc[klast, 'sltp_pini'] = self.sltp_pini 
                    # register price when the position ends due to sltp
                    self.data.loc[klast, 'sltp_pend'] = self.data.Close.loc[klast]
                    # create a list with all the prices when sltp was triggered
                    self.sltp_pend_list.append(self.data.Close.loc[klast])
                    self.execute_trades()
                    self.calculate_returns()
                    self.update_balance_sc()   

            # check if position in previous kandle is long and if so, analize sltp condition
            if (self.data.position.loc[kprev] == 1):
                # condition for take profit in long position            
#                 print('tp_flag_long assessment')
#                 print('self.data.Close.loc[klast]', self.data.Close.loc[klast])
#                 print('self.sltp_pini', self.sltp_pini)
#                 print('self.sltp_pini * (1 + self.tp)', self.sltp_pini * (1 + self.tp))
#                 print('self.data.Close.loc[klast] >= self.sltp_pini * (1 + self.tp)', self.data.Close.loc[klast] >= self.sltp_pini * (1 + self.tp))             
#                 print(((self.data.Close.loc[klast]/self.sltp_pini)-1), self.tp)
                self.tp_flag_long = self.data.Close.loc[klast] >= self.sltp_pini * (1 + self.tp)  
#                 print('self.tp_flag_long', self.tp_flag_long)                   
                
                # condition for stop loss in long position               
#                 print('sl_flag_long assessment')
#                 print('self.data.Close.loc[klast]', self.data.Close.loc[klast])
#                 print('self.sltp_pini', self.sltp_pini)
#                 print('self.sltp_pini * (1 - self.sl)', self.sltp_pini * (1 - self.sl))
#                 print('self.data.Close.loc[klast] <= self.sltp_pini * (1 - self.sl)', self.data.Close.loc[klast] <= self.sltp_pini * (1 - self.sl))                 
#                 print(((self.data.Close.loc[klast]/self.sltp_pini)-1), self.sl)
                self.sl_flag_long = self.data.Close.loc[klast] <= self.sltp_pini * (1 - self.sl)    
#                 print('self.sl_flag_long', self.sl_flag_long)                  
                
                if self.tp_flag_long:
                    self.data.loc[klast, 'tp_flag_long'] = 'tp_flag_long'             
                    self.sltp_flag = True                
                elif self.sl_flag_long:
                    self.data.loc[klast, 'sl_flag_long'] = 'sl_flag_long'
                    self.sltp_flag = True               
                else:
                    self.sltp_flag = False # if no sltp condition it is necessary to reset this variable to not jumpt accidentaly into executing a new trade        

                if (self.sltp_flag):           
                    self.data.loc[klast, 'position'] = 0
                    self.execute_trades()
                    self.calculate_returns()
                    self.update_balance_sc()     
                    
            # if no sltp signal is triggered then do nothing
        except Exception as e:
            self.handle_exception(e, 'assess_sltp')
              
    def update_position(self):
        '''
        Update position based on updated technical indicators values
        '''
        try:
            # flag to mark that the current position has not been stablished by prepare_recent_data
            self.is_initial = False

            # if sltp is triggered by assess_sltp do not update the position for self.klast kandle since it has been already updated
            # and wait for the next kandle

            if (self.sltp_flag):
                self.sltp_flag = False
                self.tp_flag_short = False
                self.sl_flag_short = False
                self.tp_flag_long = False
                self.sl_flag_long = False             
                return

            klast = self.klast #
            kprev = self.kprev #

            # --------------------- MACD CROSS SIGNAL + SMA CONFIRMATION ---------------------

            # macd cross long
            ht_macd_diff_neg = self.data.macd_diff.loc[kprev] < 0 #
            ht_macd_diff_plusone_pos = self.data.macd_diff.loc[klast] > 0 #   
            # macd cross short
            ht_macd_diff_pos = self.data.macd_diff.loc[kprev] > 0 #
            ht_macd_diff_plusone_neg = self.data.macd_diff.loc[klast] < 0 #

            if (ht_macd_diff_neg and ht_macd_diff_plusone_pos): #
                self.data.loc[klast, 'macd_inv_sign'] = 1 #
            elif (ht_macd_diff_pos and ht_macd_diff_plusone_neg): #
                self.data.loc[klast, 'macd_inv_sign'] = -1 #e
            else: #
                self.data.loc[klast, 'macd_inv_sign'] = 0 #

            # sma long confirmation
            sma_conf_long = self.data.ema_trend_fast_ps.loc[klast] > self.data.sma_trend_slow_ps.loc[klast] #
            # sma short confirmation
            sma_conf_short = self.data.ema_trend_fast_ps.loc[klast] < self.data.sma_trend_slow_ps.loc[klast] #
            if sma_conf_long: #
                self.data.loc[klast, 'sma_conf'] = 1 #
            elif sma_conf_short: #
                self.data.loc[klast, 'sma_conf'] = -1 #
            else: #
                self.data.loc[klast, 'sma_conf'] = 0 #

            # ---------------------------------------------------------------------------------

            # --------------------- SMA CROSS SIGNAL + MACD_DIFF CONFIRMATION -----------------

            # sma cross long
            ht_sma_neg = self.data.ema_trend_fast_ps.loc[kprev] < self.data.sma_trend_slow_ps.loc[kprev] #
            ht_sma_plusone_pos = self.data.ema_trend_fast_ps.loc[klast] > self.data.sma_trend_slow_ps.loc[klast] #        
            # sma cross short
            ht_sma_pos = self.data.ema_trend_fast_ps.loc[kprev] > self.data.sma_trend_slow_ps.loc[kprev] #
            ht_sma_plusone_neg = self.data.ema_trend_fast_ps.loc[klast] < self.data.sma_trend_slow_ps.loc[klast] #

            if (ht_sma_neg and ht_sma_plusone_pos): #
                self.data.loc[klast, 'sma_inv_sign'] = 1 #
            elif (ht_sma_pos and ht_sma_plusone_neg):
                self.data.loc[klast, 'sma_inv_sign'] = -1 #
            else: #
                self.data.loc[klast, 'sma_inv_sign'] = 0 #                                                              

            # macd_diff short confirmation
            macd_diff_conf_short = self.data.loc[klast, 'macd_diff'] < 0 #
            # macd_diff long confirmation
            macd_diff_conf_long = self.data.loc[klast, 'macd_diff'] > 0 #

            if macd_diff_conf_long: #
                self.data.loc[klast, 'macd_diff_conf'] = 1 #
            elif macd_diff_conf_short: #
                self.data.loc[klast, 'macd_diff_conf'] = -1 #
            else:
                self.data.loc[klast, 'macd_diff_conf'] = 0 #

            # -------------------------------- POSITION DEFINITION -----------------------------
            condlong1 = ((self.data.loc[klast, 'macd_inv_sign'] == 1 and self.data.loc[klast, 'sma_inv_sign'] == 1) or (self.data.loc[klast, 'macd_inv_sign'] == 1 and self.data.loc[klast, 'sma_conf'] == 1))
            condshort1 = ((self.data.loc[klast, 'macd_inv_sign'] == -1 and self.data.loc[klast, 'sma_inv_sign'] == -1) or (self.data.loc[klast, 'macd_inv_sign'] == -1 and self.data.loc[klast, 'sma_conf'] == -1))     
            condlong2 = ((self.data.loc[klast, 'sma_inv_sign'] == 1 and self.data.loc[klast, 'macd_inv_sign'] == 1) or (self.data.loc[klast, 'sma_inv_sign'] == 1 and self.data.loc[klast, 'macd_diff_conf'] == 1))
            condshort2 = ((self.data.loc[klast, 'sma_inv_sign'] == -1 and self.data.loc[klast, 'macd_inv_sign'] == -1) or (self.data.loc[klast, 'sma_inv_sign'] == -1 and self.data.loc[klast, 'macd_diff_conf'] == -1))

            if (condlong1 == True or condlong2 == True and condshort1 == False and condshort2 == False):
                self.data.loc[klast, 'position'] = 1

            elif (condshort1 == True or condshort2 == True and condlong1 == False and condlong2 == False):
                self.data.loc[klast, 'position'] = -1        

            elif (condlong1 == False and condshort1 == False and condlong2 == False and condshort2 == False):
                if (self.data.loc[kprev, 'position'] == 1):
                    self.data.loc[klast, 'position'] = 1

                elif (self.data.loc[kprev, 'position'] == -1):
                    self.data.loc[klast, 'position'] = -1

                else:
                    self.data.loc[klast, 'position'] = 0      
            else:    
                print(f"Unexpected branch assessment in update_position() method at time={klast}")

            self.calculate_returns()  
        except Exception as e:
            self.handle_exception(e, 'update_position')
    
    def execute_trades(self):
        '''
        Execute trades if necessary conditions are met'''
        try:
            klast = self.klast
            tevent = self.tevent

            if self.data["position"].iloc[-1] == 1: # if position is long -> go/stay long
                if self.position == 0:
                    order = self.client.futures_create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quantity = self.units)
                    self.position = 1
                    self.assess_order_status(order, "GOING LONG") 
                elif self.position == -1:
                    order = self.client.futures_create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quantity = 2 * float(self.units))
                    self.position = 1
                    self.assess_order_status(order, "GOING LONG")

            elif self.data["position"].iloc[-1] == 0: # if position is neutral -> go/stay neutral
                if self.position == 1:
                    if (self.tp_cont == True and self.sltp_tini != None and self.tp_flag_long == True): #if tp_cont the go neutral + long
                        order = self.client.futures_create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                        self.position = 0
                        self.assess_order_status(order, "GOING NEUTRAL")                    
                        order = self.client.futures_create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quantity = self.units)
                        self.position = 1
                        self.assess_order_status(order, "GOING LONG")
                        self.data.loc[klast, 'position'] = 1 # 'bypass' update of the lastest position
                        self.calculate_returns()
                        self.update_balance_sc()
                    else: #if tp_cont != True then only go neutral
                        order = self.client.futures_create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                        self.position = 0
                        self.assess_order_status(order, "GOING NEUTRAL")    
                        self.sltp_tini = None        
                elif self.position == -1:              
                    if (self.tp_cont == True and self.sltp_tini != None and self.tp_flag_short == True):
                        order = self.client.futures_create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quantity = self.units)
                        self.position = 0
                        self.assess_order_status(order, "GOING NEUTRAL")                  
                        order = self.client.futures_create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                        self.position = -1
                        self.assess_order_status(order, "GOING SHORT")
                        self.data.loc[klast, 'position'] = -1 # 'bypass' update of the lastest position
                        self.calculate_returns()
                        self.update_balance_sc()                    
                    else:
                        order = self.client.futures_create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quantity = self.units)
                        self.position = 0
                        self.assess_order_status(order, "GOING NEUTRAL")
                        self.sltp_tini = None
            elif self.data["position"].iloc[-1] == -1: # if position is short -> go/stay short
                if self.position == 0:
                    order = self.client.futures_create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                    self.position = -1 
                    self.assess_order_status(order, "GOING SHORT")
                elif self.position == 1:
                    order = self.client.futures_create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = 2 * float(self.units))
                    self.position = -1
                    self.assess_order_status(order, "GOING SHORT")
        except Exception as e:
            self.handle_exception(e, 'execute_trades')


    def assess_order_status(self, order=None, GOING=None):
        '''
        Ensure that the executed order has been filled
        '''
        try: 
            time.sleep(3) # ensure that the order is created and that is going to be found
            order_details = self.client.futures_get_order(symbol = self.symbol, orderId = order["orderId"]) # check order status
            print(order_details)
            if (self.position != 0):
                order_time = order["updateTime"]
                order_time_micro = pd.to_datetime(order_time, unit = "ms")
                order_time = order_time_micro.replace(microsecond=0)  
                self.sltp_tini = order_time   
                self.sltp_pini = round(float(order_details['avgPrice']),3)
            self.orders_list.append(order_details)           
            self.report_trade(order, GOING)
            if (order_details['status'] != 'FILLED'):
                raise Exception('Not filled order')
        except Exception as e:
            self.handle_exception(e, 'assess_order_status')
    
    def report_trade(self, order, going):
        '''
        Report executed trade details
        '''
        try:
            self.orders += 1
            order_time = order["updateTime"]
            trades = self.client.futures_account_trades(symbol = self.symbol, startTime = order_time)
        except Exception as e:
            self.handle_exception(e, 'report_trade')
        
        order_time_micro = pd.to_datetime(order_time, unit = "ms")
        order_time = order_time_micro.replace(microsecond=0)
        self.trades_list.append(trades)
        # extract data from order object
        df = pd.DataFrame(trades)
        msg3 = ''
        if (len(df) == 0):
            msg3 = 'info of trades list not available'
        else:
            self.last_order = df
            columns = ["qty", "quoteQty", "commission","realizedPnl"]
            for column in columns:
                df[column] = pd.to_numeric(df[column], errors = "coerce")
            base_units = round(df.qty.sum(), 5)
            quote_units = round(df.quoteQty.sum(), 5)
            commission = -round(df.commission.sum(), 5)
            real_profit = round(df.realizedPnl.sum(), 5)
            price = round(quote_units / base_units, 5)
            self.cum_profits += round((commission + real_profit), 5)
            msg5 = "Base_Units = {} | Quote_Units = {} | Price = {}".format(base_units, quote_units, price)
           
        # update balance taking into account comisions
        self.update_balance_sc()    
        # print trade report
        print(2 * "\n" + 100* "-")
        msg1 = "{} | {}".format(order_time, going)
        print(msg1) 
        msg2 = "{} | Stable coin  = {} | Bal. Profit. Act. = {} | Bal. Profit. End. = {} ".format(order_time, self.sc_name, self.balance_profit_actual, self.balance_profit_end)
        print(msg2)
        if (len(msg3)>0):
            print(msg3)
        msg4 = "stop loss % = {} | take profit % = {} | Interval = {} | Symbol = {} | Leverage = {} | tp_cont = {}".format(self.sl, self.tp, self.interval, self.symbol, self.leverage, self.tp_cont)
        print(msg4)
        print(msg5)
        print(100 * "-" + "\n")
        mail_msg = f"Subject: trade executed. \n\n {msg1} \n\n {msg2} \n\n {msg3} \n\n {msg4} \n\n {msg5}"
              
        try:
            self.conn.sendmail('jpxcar6@gmail.com', 'jpxcar6@gmail.com', f"{mail_msg}")
        except (smtplib.SMTPSenderRefused, smtplib.SMTPException, smtplib.SMTPServerDisconnected, smtplib.SMTPResponseException) as e:
            print(e)
            self.login_mail()
            self.conn.sendmail('jpxcar6@gmail.com', 'jpxcar6@gmail.com', f"{mail_msg}")

    def update_balance_sc(self):
        '''
        Update wallet balance
        '''
        try:
            klast = self.klast
            balance_list = self.client.futures_account_balance()
            self.balance_actual = self.get_actual_balance()
            for item in balance_list:
                if item['asset'] == self.sc_name:
                    self.balance_sc_raw = item['balance']
                    self.data.loc[klast, 'balance_sc'] = round(float(self.balance_sc_raw),3)
                    self.balance_profit_actual = round((self.balance_actual - self.balance_initial),3)
                    print(self.data.loc[klast, 'balance_sc'])
        except Exception as e:
            self.handle_exception(e, 'update_balance_sc')              
    
    def get_actual_balance(self):
        '''
        Get current wallet balance
        '''
        try:
            balance_list = self.client.futures_account_balance()
            for item in balance_list:
                if item['asset'] == self.sc_name:
                    return(round(float(item['balance']),3))
        except Exception as e:
            self.handle_exception(e, 'get_actual_balance')
              
    def end_socket(self):
        '''
        End socket function
        '''
        try:
            self.twm.stop()
        except Exception as e:
            self.handle_exception(e, 'end_socket')              
    
    def stop_ses(self, save_to_file=True):
        '''
        To stop session when conditions are met
        '''
        print('stop ses')
        try:      
            self.run_end_time_utc = datetime.utcnow()
            dt = self.run_end_time_utc - self.trade_start_time_utc
            self.run_end_delta = round(dt.seconds/60,0)
            print(f"trading sesion duration = {self.run_end_delta} minutes up to {self.assigned_duration_minutes}")

            # stablish neutral position before kandle is complete. Handle the position logic in execute_trades()
            klast = self.klast
            self.data.loc[klast, 'position'] = 0
            self.calculate_returns()
            self.balance_end = self.get_actual_balance()
            self.balance_profit_end = round((self.balance_end - self.balance_initial),3)        
            if (self.orders != 0):
                self.execute_trades()           
            else:
                pass
            print('SESION STOPPED')
            self.twm.stop()

            if (save_to_file == True):
                self.save_to_files()
        except Exception as e:
            self.handle_exception(e, 'stop_ses') 
              
    def plot_results(self, start_plot=None, end_plot=None, width_bars=0.1):
        '''
        Plot sesions results
        '''
        try:
            data_ready_raw = self.data.dropna(subset=['macd_diff', 'macd_macd', 'macd_signal', 'ema_trend_fast_ps', 'sma_trend_slow_ps', 'multiple_hold_acum', 'macd_simple_ret_net_lev_acum', 'macd_multiple_net_acum']).copy()
            data_ready = data_ready_raw[self.trade_start_time_utc-self.td:]

            if (len(data_ready) < 2):
                return 'No enough data yet to create a plot'        

            colors=[]

            fig, (close_ax, macd_ax, acum_ax, balance_ax_sc) = plt.subplots(nrows=4, ncols=1, figsize=(30,20), sharex=True)

            close_ax.grid(visible=True, which='major', axis='x', color='grey')
            macd_ax.grid(visible=True, which='major', axis='x', color='grey')
            close_ax.grid(visible=True, which='major', axis='y', color='grey')
            macd_ax.grid(visible=True, which='major', axis='y', color='grey')
            close_ax.grid(visible=True, which='minor', axis='x', color='grey')
            macd_ax.grid(visible=True, which='minor', axis='x', color='grey')

            close_ax.tick_params(labelrotation=45, labelsize = 'large')
            macd_ax.tick_params(labelrotation=45, labelsize = 'large')

            close_ax.margins(0)
            macd_ax.margins(0)

            close_ax.set_ylim(auto=True)

            if (start_plot == None):
                start_plot = data_ready.index[0]
            if (end_plot == None):
                end_plot = data_ready.index[-1]  

            if ((start_plot != None) and (end_plot !=None)):
                cond_start = data_ready.index >= start_plot
                cond_end = data_ready.index <= end_plot
                data_ready = data_ready[cond_start&cond_end]

            for index, value in data_ready.macd_diff.iteritems():
                if value > 0:
                    colors.append('g')
                else:
                    colors.append('r')

            close_ax.plot(data_ready.index, data_ready.Close) #plot the data without shifting
            close_ax.plot(data_ready.index, data_ready.ema_trend_fast_ps)
            close_ax.plot(data_ready.index, data_ready.sma_trend_slow_ps)
            close_ax.legend(['Close', 'ema_trend_fast_ps', 'sma_trend_slow_ps'])

            long_pos = data_ready.position == 1              
            long_trade = data_ready.loc[long_pos]
            short_pos = data_ready.position == -1            
            short_trade = data_ready.loc[short_pos]
            neutral_pos = data_ready.position == 0           
            neutral_trade = data_ready.loc[neutral_pos]        
            close_ax.scatter(short_trade.index, short_trade.Close.loc[short_trade.index], marker='^', color='r', s=100)
            close_ax.scatter(long_trade.index, long_trade.Close.loc[long_trade.index], marker='^', color='g', s=100)
            close_ax.scatter(neutral_trade.index, neutral_trade.Close.loc[neutral_trade.index], marker='^', color='b', s=100)
            close_ax.scatter(self.tevent_list, self.sltp_pend_list, marker='^', color='b', s=100)
            macd_ax.bar(x= data_ready.index, height= data_ready.macd_diff, width=width_bars, align='center', color=colors, edgecolor='black')       

            if ((self.data.Complete.iloc[-1] == False) and (self.data.position.iloc[-1] == 0)):
                close_ax.scatter(self.data.index[-1], self.data.Close.loc[self.data.index[-1]], marker='^', color='r', s=100)

            if self.leverage: # NEW!
                to_analyse = data_ready.macd_simple_ret_net_lev_acum
                multiple_legend = 'macd_multiple_net_leverage'
            else: 
                to_analyse = data_ready.macd_multiple_net_acum
                multiple_legend = 'macd_multiple_net_noleverage'

            acum_ax.plot(data_ready.index, data_ready.multiple_hold_acum)
            acum_ax.plot(data_ready.index, to_analyse)  
            acum_ax.legend(['multiple_hold_acum', multiple_legend],fontsize=14) 
            balance_ax_sc.plot(data_ready.index, data_ready.balance_sc)
            balance_ax_sc.legend(['balance_sc'])
        except Exception as e:
            self.handle_exception(e, 'plot_results') 
              
    def save_to_files(self):
        '''
        Save data to files.
        '''
        try:
            file_name = f"{self.symbol}__{self.interval}__eslow_{self.ema_slow}_efast_{self.ema_fast}_esign_{self.ema_signal}__smas_{self.sma_trend_slow}__smaf_{self.ema_trend_fast}__duration_{self.run_end_delta}min_upto_{self.assigned_duration_minutes}__ordersnum_{self.orders}" 
            outfile = open(file_name, 'wb')
            self.data.to_csv(outfile, index = True, header = True, sep = ',', encoding = 'utf-8', date_format ='%Y-%m-%d-%H:%M')
            outfile.close()
            results = {
                "units": self.units,
                "symbol": self.symbol,
                "interval": self.interval,
                "ema_slow": self.ema_slow,
                "ema_fast": self.ema_fast,
                "ema_signal": self.ema_signal,
                "sma_trend_slow": self.sma_trend_slow,
                "ema_trend_fast": self.ema_trend_fast,
                "sl": self.sl,
                "tp": self.tp,
                "leverage": self.leverage,
                "testnet": self.testnet,
                "assigned_duration_minutes": self.assigned_duration_minutes,
                "run_end_time_utc": self.run_end_time_utc.strftime("%Y-%m-%d-%H:%M:%S"),
                "run_end_delta": self.run_end_delta,
                "orders": self.orders,
                "trade_start_time_utc": self.trade_start_time_utc.strftime("%Y-%m-%d-%H:%M:%S"),
                "profit": self.cum_profits,
                "balance_profit_end": self.balance_profit_end
            }
            f = open(f"{file_name}.txt", "a")
            json.dump(results, f)
            f.close()
        except Exception as e:
            self.handle_exception(e, 'save_to_files')     

    def login_mail(self):
        '''
        login email to receive trade execution notifications
        '''
        try:
            self.conn = smtplib.SMTP('smtp.gmail.com', 587)   
            self.conn.ehlo()
            self.conn.starttls()
            self.conn.login('email', 'password')
        except Exception as e:
            self.handle_exception(e, 'login_mail')         
    
    def logout_mail(self):
        '''
        logout email
        '''
        try:
            self.conn.quit()
        except Exception as e:
            self.handle_exception(e, 'logout_mail')               

In [20]:
#futures_trader_instance = Futures_trader(sc_name='BUSD', api_key=keys.TESTNET_API_KEY, api_secret=keys.TESTNET_API_SECRET, testnet=True, symbol='BTCUSDT', units=0.5, interval='1m', ema_slow=26, ema_fast=2, ema_signal=12, sma_trend_slow=20, ema_trend_fast=2, sl=0.02, tp=0.06, leverage=10, assigned_duration_minutes=360, tp_cont=False)

In [19]:
#futures_trader_instance.start_trading()

In [18]:
#futures_trader_instance.stop_ses()

In [17]:
#futures_trader_instance.plot_results(width_bars=0.0005)